使用 PySpark 求总和、均值、最值与方差

## 初始化

In [1]:
from pyspark import SparkConf, SparkContext

In [2]:
conf = SparkConf().setMaster("local[*]").setAppName('My App')
sc = SparkContext(conf = conf)

## 创建数据

In [5]:
## 创建一个RDD
nums = sc.parallelize([1, 2, 3, 4, 5])

## 所有元素的总和

如果RDD中每个元素是单个值，则通过 $reduce(func(x, y)$ 传入输入参数为 $2$ 个值的函数时，它会被顺序作用在不同的分区当中。计算各分区的结果 $[y1, y2]$ 以后，再通过$func(y1, y2)$连接。

In [6]:
# 求和
nums.reduce(lambda x, y: x + y)

15

In [3]:
# 减法计算，5个数的计算顺序为 (1 - 2 - 3) - (4 - 5)
sc.parallelize([1, 2, 3, 4, 5]).reduce(lambda x, y: x - y)

-3

## 不同分区数量情况下RDD的计算

**RDD的分区数量会影响到计算的并行性**

我的硬件配置信息如下：

型号名称：	MacBook Pro

型号标识符：	MacBookPro12,1

处理器名称：	Dual-Core Intel Core i5

处理器速度：	2.7 GHz

处理器数目：	1

核总数：	2

超线程技术：	已启用

内存：	8 GB

In [4]:
# 先创建一个数组
array = [i for i in range(int(1e7))]

In [5]:
len(array)

10000000

In [11]:
# 创建单个分区RDD
rdd1 = sc.parallelize(array, 1)
# 创建两个分区的RDD
rdd2 = sc.parallelize(array, 2)
# 创建默认数量分区的RDD，在我的机器上默认为4
rdd3 = sc.parallelize(array)

In [37]:
# 单线程运行
%timeit rdd1.reduce(lambda x, y: x - y)

3.52 s ± 261 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
# 双线程运行
%timeit rdd2.reduce(lambda x, y: x - y)

1.86 s ± 88.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
# 四线程运行
%timeit rdd3.reduce(lambda x, y: x - y)

2.39 s ± 186 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## 最值

In [13]:
# 最大值
nums.reduce(lambda x, y: max(x, y))

5

In [14]:
# 最小值
nums.reduce(lambda x, y: min(x, y))

1

## 均值

In [24]:
# 平均值 = 求和 / 个数
avg = nums.reduce(lambda x, y: x + y) / nums.count()
avg

3.0

## 方差

In [25]:
nums.map(lambda x : x - avg).map(lambda x: x * x).reduce(lambda x, y: x + y)

10.0